In [2]:
import numpy as np
import pandas as pd
import re
import pickle
import os
import ast

In [3]:

files_names = ["IMV_ABA_relax_mRMR_0.out", 
               "IMV_ABA_relax_mRMR_1.out",
               "IMV_ABA_relax_mRMR_2.out", 
               "IMV_ABA_relax_mRMR_3.out",
               "IMV_ABA_relax_mRMR_4.out"]




# Function to clean and extract float values, skipping 'np.float64()' and handling nested lists
def extract_floats_from_line(line):
    # Remove any 'np.float64()' occurrences using regex
    float_values = re.findall(r'[-+]?\d*\.\d+(?:[eE][-+]?\d+)?', line)
    # Convert them to float type
    return [float(value) for value in float_values]

# Function to clean and extract float values, skipping 'np.float64()' and handling nested lists
def extract_floats_from_line_classic(line):
    # Remove any 'np.float64()' occurrences using regex
    float_values = re.findall(r'[-+]?\d*\.\d+(?:[eE][-+]?\d+)?', line)
    # Convert them to float type
    float_values = [float(value) for value in float_values]
    scores = float_values[:6]
    class_rep =  [float_values[6:][i:i+3] for i in range(0, len(float_values[6:]), 3)]
    scores.append(class_rep)
    return scores

list_of_dicc_SVM = []
list_of_class_dicc_SVM = []

list_of_dicc_KNN = []
list_of_class_dicc_KNN = []

for file_name in files_names:

    list_of_names = ["Index", "coverage", "inefficiency", 
                    "certainty", "uncertainty", "mistrust",
                    "S_score", "F_score", "Creditibily"]

    OUT_SVM = {}       
    for i in range(0, len( list_of_names)):
        OUT_SVM[list_of_names[i]] = []

    OUT_KNN = {}
    for i in range(0, len( list_of_names)):
        OUT_KNN[list_of_names[i]] = []

        
    list_of_classical_names = ['accuracy', 'precision', 'recall', 
                              'f1_micro', 'f1_macro', 'f1_weighted','per_class']

    OUT_CLASSIC_SVM = {}   
    for i in range(0, len( list_of_classical_names)):
        OUT_CLASSIC_SVM[list_of_classical_names[i]] = []


    OUT_CLASSIC_KNN = {}
    for i in range(0, len( list_of_classical_names)):
        OUT_CLASSIC_KNN[list_of_classical_names[i]] = []


# Read the content from the .txt file and process line by line
    with open(file_name, 'r') as file:
        counter = 0
        index = 0
        for line in file:

            # If the line contains 'CLASSIC' (SVM or KNN), we skip it as it's just a label
            out = extract_floats_from_line(line)
            if len(out) == 9:
                
                if counter == 1:
                    OUT_KNN["Index"] += [selected_features[index]]
                    for i in range(1, len( list_of_names)):
                        OUT_KNN[list_of_names[i]] += [out[i]]

                    counter = 0 # update the counter
                    index += 1

                else:
                    OUT_SVM["Index"] += [selected_features[index]]
                    for i in range(1, len( list_of_names)):
                        OUT_SVM[list_of_names[i]] += [out[i]]

                    counter += 1 # update the counter

            if 'SVM' in line:
                print(line)
                # Extract floats from the current line
                floats_in_line = extract_floats_from_line_classic(line)
                # Print the extracted floats from the line
                print(f"{floats_in_line}")

                for i in range(0, len( list_of_classical_names)):
                    OUT_CLASSIC_SVM[list_of_classical_names[i]] += [floats_in_line[i]]

            if 'KNN' in line:
                print(line)
                # Extract floats from the current line
                floats_in_line = extract_floats_from_line_classic(line)
                # Print the extracted floats from the line
                print(f"{floats_in_line}") 

                for i in range(0, len( list_of_classical_names)):
                    OUT_CLASSIC_KNN[list_of_classical_names[i]] += [floats_in_line[i]]

            if line[0] == '[':
                selected_features = ast.literal_eval(line)
                

        
    list_of_dicc_SVM.append(OUT_SVM)
    list_of_class_dicc_SVM.append(OUT_CLASSIC_SVM)

    list_of_dicc_KNN.append(OUT_KNN)
    list_of_class_dicc_KNN.append(OUT_CLASSIC_KNN)


with open(os.path.join("IMV_ABA_relax_mRMR_conformal_SVM.pickle"), 'wb') as f:
    pickle.dump(list_of_dicc_SVM, f)

with open(os.path.join("IMV_ABA_relax_mRMR_classic_SVM.pickle"), 'wb') as f:
    pickle.dump(list_of_class_dicc_SVM, f)


with open(os.path.join("IMV_ABA_relax_mRMR_conformal_KNN.pickle"), 'wb') as f:
    pickle.dump(list_of_dicc_KNN, f)

with open(os.path.join("IMV_ABA_relax_mRMR_classic_KNN.pickle"), 'wb') as f:
    pickle.dump(list_of_class_dicc_KNN, f)




SVM CLASSIC:  [0.35714285714285715, np.float64(0.2383333333333333), np.float64(0.3222222222222222), np.float64(0.35714285714285715), np.float64(0.2739756367663344), np.float64(0.3044138585666825), [[0.475, 0.6333333333333333, 0.5428571428571428], [0.0, 0.0, 0.0], [0.24, 0.3333333333333333, 0.27906976744186046]]]

[0.35714285714285715, 0.2383333333333333, 0.3222222222222222, 0.35714285714285715, 0.2739756367663344, 0.3044138585666825, [[0.475, 0.6333333333333333, 0.5428571428571428], [0.0, 0.0, 0.0], [0.24, 0.3333333333333333, 0.27906976744186046]]]
KNN CLASSIC:  [0.4714285714285714, np.float64(0.4384645515080298), np.float64(0.4286195286195286), np.float64(0.4714285714285714), np.float64(0.42271183763721076), np.float64(0.45387213591904424), [[0.5675675675675675, 0.7, 0.6268656716417911], [0.34782608695652173, 0.36363636363636365, 0.35555555555555557], [0.4, 0.2222222222222222, 0.2857142857142857]]]

[0.4714285714285714, 0.4384645515080298, 0.4286195286195286, 0.4714285714285714, 0.422

In [5]:
import os
import pandas as pd
import numpy as np
import itertools
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
#from matplotlib_venn import venn3,venn2

import pickle
import argparse




def plot_comparison(OUT, list_of_names, N_classes):

    # read the dictionary from the file
    with open( OUT +'.pickle', 'rb') as f:
        OUT = pickle.load(f)



    data_mean = {}
    data_std = {}

    for i in range(0, len( list_of_names)):
        data_mean[list_of_names[i]] = []
        data_std[list_of_names[i]] = []

    for ele in OUT:

        keys = list(ele.keys())
        values = list(ele.keys())

        x = [*range(1 , len(ele["inefficiency"]) + 1)]

        #for i in range(len(values[0])):

        for name in list_of_names:
            
            data_mean[name] += [ele[name]]
            print(ele[name])
            #if name != "inefficiency":
                
            #    data_mean[name] += [ele[name]]
            #else:
                
            #    data_mean[name] += [MinMaxscaler(ele[name], 1, N_classes)]
                
    return data_mean, data_std, x



dataset_user = "IMV_ABA"
N_of_classes = 3
n_features = 50
methods = ["JMI"]
classifiers = ["SVM"]

#save_path = 'save/save_'+ dataset_user + '/'+ "Best_conformal_scores"+ dataset_user + ".csv"



dataset = dataset_user

list_of_names = [ "inefficiency", "P", "certainty", "P", "uncertainty", "P", "mistrust", "P",]

df = pd.DataFrame(columns=list_of_names)

list_of_names = [ "coverage","inefficiency", "certainty",  "uncertainty",  "mistrust", "S_score", "F_score", "Creditibily"]

for classifier in classifiers:

    for method in methods:

        method_file = "../"+ dataset +"_" +  method + "_conformal_" + classifier 

        #"..\mRMR-MS_new\mRMR-MS_new\save\save_breast\\breast_mRMR-MS_tangent_conformal_XGBOOST"

    


        if dataset == "subiculum":
            subi = 1
        else:
            subi = 0


        data_mean, data_std, x_1 = plot_comparison(method_file, list_of_names, N_of_classes )
        


        list_of_values = []
        for name in list_of_names:

            if name in ["coverage", "S_score", "F_score", "Creditibily"]:
                continue

        
            mean = np.mean(np.array(data_mean[name]), axis=0).tolist()
            std_dev = np.std(np.array(data_mean[name]), axis=0).tolist()

        
    
            alfa = 0.05

            if name == "inefficiency": 

                if subi == 1:
                # Specify the target number
                    target_number = 1.0

                    mean=np.array(mean)
            
            
                    nearest_value = mean[np.where(mean<=1.0)[0][0]-1]
                    nearest_position = np.where(mean<=1.0)[0][0]-1


                    #print(name, ": ", round(nearest_value,2), nearest_position + 1)
                    list_of_values.append(round(nearest_value,2))
                    list_of_values.append(int(nearest_position + 1))

                else:
                    #print(name, ": ", round(np.min(mean),2), np.argmin(mean) + 1)
                    list_of_values.append(round(np.min(mean),2))
                    list_of_values.append(int(np.argmin(mean) + 1))

                    #t_stat, p_valor = stats.ttest_ind(mean_1, mean_2, alternative = "less")
    
            elif name == "certainty":
                #print(name, ": ", round(np.max(mean),2), np.argmax(mean)+1)
                list_of_values.append(round(np.max(mean),2))
                list_of_values.append(int(np.argmax(mean)+1))
                #t_stat, p_valor = stats.ttest_ind(mean_1, mean_2,  alternative = "less")

            elif name == "uncertainty":
                #t_stat, p_valor = stats.ttest_ind(mean_1, mean_2, alternative = "less")
                #print(name, ": ", round(np.min(mean),2), np.argmin(mean)+1)
                list_of_values.append(round(np.min(mean),2))
                list_of_values.append(int(np.argmin(mean)+1))

            elif name == "mistrust":
                #print(name, ": ", round(np.max(mean),2), np.argmax(mean)+1)
                list_of_values.append(round(np.max(mean),2))
                list_of_values.append(int(np.argmax(mean)+1))
                #t_stat, p_valor = stats.ttest_ind(mean_1, mean_2, alternative = "less")

            else:
                pass
        
        new_series = pd.Series(list_of_values, index=df.columns, name=method + "-" + classifier)

        # Concatenar la nueva serie al DataFrame
        df = pd.concat([df, new_series.to_frame().T], ignore_index=False)
    

print(df)

[0.9142857142857143, 0.9571428571428572, 0.9, 0.9428571428571428, 0.9571428571428572, 0.9428571428571428, 0.9428571428571428, 0.9285714285714286, 0.9285714285714286, 0.9428571428571428, 0.9285714285714286, 0.9, 0.9, 0.8571428571428571, 0.8857142857142857, 0.9, 0.8857142857142857, 0.9, 0.9, 0.8857142857142857, 0.8857142857142857, 0.8857142857142857, 0.9428571428571428, 0.9285714285714286, 0.9285714285714286, 0.9, 0.9285714285714286, 0.8857142857142857, 0.8428571428571429, 0.8571428571428571, 0.8571428571428571, 0.8428571428571429, 0.8571428571428571, 0.8, 0.8285714285714286, 0.8428571428571429, 0.8285714285714286, 0.8285714285714286, 0.8142857142857143, 0.8285714285714286, 0.9, 0.9, 0.9, 0.9142857142857143, 0.9, 0.8571428571428571, 0.8571428571428571, 0.8428571428571429, 0.8714285714285714, 0.8142857142857143]
[2.742857142857143, 2.857142857142857, 2.7285714285714286, 2.742857142857143, 2.6, 2.6, 2.6285714285714286, 2.585714285714286, 2.6, 2.6, 2.657142857142857, 2.557142857142857, 2.6,